In [4]:
import csv
import json
import os
import requests
import urllib3


def make_url(url, params):

    temp = '?'
    count = 0
    for i in params:
        temp = temp + i + '=' + params[i]
        count = count + 1

        if count != (len(params)):    
            temp = temp + '&'

    return url + temp


def make_csv(data, fileName, info):
    if(info == 'location_info'):

        data = json.loads(data)

        items = data["response"]["body"]["items"]["item"]
        filtered_data = [{"code": item["code"], "name": item["name"]} for item in items]

        csv_file_path = f'{fileName}.csv'

        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:

            csv_writer = csv.DictWriter(csv_file, fieldnames=["code", "name"])
            
            csv_file.seek(0, 2)
            file_empty = csv_file.tell() == 0
            
            if file_empty:
                csv_writer.writeheader()
            
            for row in filtered_data:
                csv_writer.writerow(row)
    elif(info == 'location_search_info'):

        data = json.loads(data)

        items = data["response"]["body"]["items"]["item"]
        parsed_list = []

        # 리스트에 필요한 데이터 추가
        for item in items:
            parsed_item = {
                "title": item["title"],
                "tel": item["tel"],
                "mapx": item["mapx"],
                "mapy": item["mapy"],
                "dist": item["dist"],           # 중심좌표로부터거리
                "cat1": item["cat1"],
                "cat2": item["cat2"],
                "cat3": item["cat3"],
                "addr1": item["addr1"],
                "addr2": item["addr2"],
                "booktour": item["booktour"],
                "contentid": item["contentid"]
            }
            parsed_list.append(parsed_item)

        # CSV 파일로 저장
        csv_file_path = f'{fileName}.csv'

        # CSV 파일 열기 (쓰기 모드)
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            # CSV 작성자 생성
            csv_writer = csv.DictWriter(csv_file, fieldnames=["contentid", "title", "addr1", "addr2", "tel", "mapx", "mapy", "dist", "cat1", "cat2", "cat3", "booktour"])
            
            # 헤더 작성
            csv_writer.writeheader()
            
            # 데이터 작성
            for row in parsed_list:
                csv_writer.writerow(row)

In [5]:
url = "https://apis.data.go.kr/B551011/KorService1/locationBasedList1"

#관광 타입 ID
contentTypeId = '15'


params = {
    #필수
    'serviceKey' : "QkiT9isHn%2FngYj7SJNBftK290c7SSWktGzgux4c8T%2Bu5%2BHox2jrUAqrY%2BdIjzM9XMSYrqLuzpU5V%2BvqTyvIs8Q%3D%3D",
    'MobileOS' : "ETC",
    'MobileApp' : "AppTest",
    'mapX' : '126.91611', # 나중에 INPUT으로 할꺼
    'mapY' : '37.568477',
    'radius' : '1000'

    # #선택
    # '_type' : 'json',
    # 'contentTypeId' : contentTypeId,
    # 'numOfRows' : '10',
    # 'pageNo' : '1'
}

In [6]:
response = requests.get(make_url(url, params), verify=False)
response

/opt/miniconda3/envs/ds_study/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [7]:
if response.status_code == 200:
    make_csv(response.text, '위치기반_관광정보_조회', 'location_search_info')
else:
    print(f'Error: {response.status_code}')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)